In [ ]:
import numpy as np
from LSTM_Learning_Lib import Model
from FeatureSetCalculation_Lib import ComputeMultiLevelLogsig1dBM
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import ParameterGrid
from sklearn import preprocessing
import random
from GetSeqMnistData import GetSeqPenandCalLogSig, GetSeqPenDigit,GetSeqPenNormCalLogSig,GetSeqPenNorm

## Hyperparameters 

In [ ]:
# Parameters grid
param_grid = {'deg_of_sig': [2,3,4,5,6], 'number_of_segment': [8],
'learning_rate': [0.001]}
Param = list(ParameterGrid(param_grid))
# Parameters
training_iters = 60
batch_size = 200

sig_comp_time = []
test_result = []
test_time = []

## train RNN with different feature sets 

In [ ]:
for i in range(np.size(Param)):
    # Raw data feature set generator
    if Param[i]['deg_of_sig']==0:
        start = time.time()
        train_X, train_Y = GetSeqPenDigit('pendigits-orig.tra.txt')
        test_X, test_Y = GetSeqPenDigit('pendigits-orig.tes.txt')
        trainsampleClip = len(train_Y)
        testsampleClip = len(test_Y)
        max_interval = 0
        for j in range(trainsampleClip):
            if max_interval < len(train_X[j]):
                max_interval = len(train_X[j])
 
        for j in range(testsampleClip):
            if max_interval < len(test_X[j]):
                max_interval = len(test_X[j])

        n_input = int(max_interval/Param[i]['number_of_segment'])+1
        if n_input % 2 != 0:
            n_input += 1
        train_data = np.zeros((trainsampleClip, n_input*Param[i]['number_of_segment']))
        test_data = np.zeros((testsampleClip, n_input*Param[i]['number_of_segment']))
        for sn in range(trainsampleClip):
            tmplen = len(train_X[sn])
            train_data[sn, :tmplen] = train_X[sn][:]
        for sn in range(testsampleClip):
            tmplen = len(test_X[sn])
            test_data[sn, :tmplen] = test_X[sn][:]
        train_data = train_data.reshape(trainsampleClip, Param[i]['number_of_segment'], n_input)
        test_data = test_data.reshape(testsampleClip, Param[i]['number_of_segment'], n_input)
        elapsed = time.time()-start
        sig_comp_time.append(elapsed)
        model3 = Model( Param[i]['learning_rate'], training_iters, batch_size, n_input, Param[i]['number_of_segment'], Param[i]['deg_of_sig'], train_data, train_Y, test_data, test_Y)
    # Folded raw data feature set generator
    elif Param[i]['deg_of_sig']==1:
        start = time.time()
        train_X, train_Y = GetSeqPenDigit('pendigits-orig.tra.txt')
        test_X, test_Y = GetSeqPenDigit('pendigits-orig.tes.txt')
        trainsampleClip = len(train_Y)
        testsampleClip = len(test_Y)
        max_interval = 0
        
        train_increment = [[] for k in range(trainsampleClip)]
        test_increment = [[] for k in range(testsampleClip)]
        for x in train_X:
            if max_interval < len(x):
                max_interval = len(x)
        for x in test_X:
            if max_interval < len(x):
                max_interval = len(x)
        print(max_interval)
        train_data = np.zeros((trainsampleClip, max_interval))
        test_data = np.zeros((testsampleClip, max_interval))
        for sn in range(trainsampleClip):
            tmplen = len(train_X[sn])
            train_data[sn, :tmplen] = train_X[sn][:]
        for sn in range(testsampleClip):
            tmplen = len(test_X[sn])
            test_data[sn, :tmplen] = test_X[sn][:]
        n_input = 2
        train_data = train_data.reshape(trainsampleClip, int(max_interval/2), 2)
        test_data = test_data.reshape(testsampleClip, int(max_interval/2), 2)
        print(train_data[0])
        elapsed = time.time()-start
        sig_comp_time.append(elapsed)
        model3 = Model( Param[i]['learning_rate'], training_iters, batch_size,  n_input, int(max_interval/2), Param[i]['deg_of_sig'], train_data, train_Y, test_data, test_Y)

    # Logsig feature set generator
    else:
        start = time.time()
        X_logsig_start, Y = GetSeqPenandCalLogSig(Param[i]['deg_of_sig'], Param[i]['number_of_segment'],'pendigits-orig.tra.txt')
        test_X_logsig_start, test_Y = GetSeqPenandCalLogSig(Param[i]['deg_of_sig'], Param[i]['number_of_segment'],'pendigits-orig.tes.txt')
        print(X_logsig_start.shape)
        print(test_X_logsig_start.shape)
        n_input = np.shape(X_logsig_start)[2]
        # number_of_samples = np.shape(X_logsig_start)[0]
        elapsed = time.time()-start
        sig_comp_time.append(elapsed)
        model3 = Model( Param[i]['learning_rate'], training_iters, batch_size, n_input, Param[i]['number_of_segment'], Param[i]['deg_of_sig'], X_logsig_start, Y, test_X_logsig_start, test_Y)
    
    # Model built and train
    fixed_error_result_model3 = model3.BuildModelKerasMn()

    print("Time = " + str(time.time()-start))
    print("Testing loss = " + str(fixed_error_result_model3['Loss']))
    # model3.KerasPredict()
    test_result.append(fixed_error_result_model3 ['Loss'])
    test_time.append(fixed_error_result_model3 ['Time'])
    
    # results save
    np.save('error_tol'+str(error_tol)+'deg_logsig'+str(Param[i]['deg_of_sig'])+'_test_result', test_result)
    np.save('error_tol'+str(error_tol)+'deg_logsig'+str(Param[i]['deg_of_sig'])+'_test_time', test_time)
    np.save('error_tol'+str(error_tol)+'deg_logsig'+str(Param[i]['deg_of_sig'])+'_sig_comp_time', elapsed)

In [ ]:
print(test_time)
print(test_result)
print(sig_comp_time)